In [10]:
import pandas as pd

train_data = pd.read_csv("data\main\df_train_b1.csv",index_col=0)


In [11]:
df_train = train_data.iloc[:, 0:17]
df_train.describe()


,label,basic_1,basic_2,basic_3,basic_4,basic_5,basic_6,basic_7,basic_8,basic_9,basic_10,basic_11,basic_12,basic_13,basic_14,basic_15,loan1_1
count,17999.000000,17999.00000,17999.000000,17999.000000,17999.000000,17999.000000,17999.000000,17999.00000,17999.0,17999.0,17999.000000,17999.0,17999.000000,17999.000000,17999.000000,17999.0,17999.000000
mean,0.033891,-83.13323,-63.085394,-19.020638,5.275127,0.566031,0.805156,40.16262,0.0,0.0,1.719818,0.0,0.000667,-79.459026,1.262237,-99.0,-0.064445
std,0.180953,36.63750,97.374722,332.683845,394.146047,9.457484,9.487904,7.79562,0.0,0.0,10.304256,0.0,0.027882,47.864356,0.439863,0.0,61.692539
min,0.000000,-99.00000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,20.00000,0.0,0.0,-99.000000,0.0,0.000000,-99.000000,1.000000,-99.0,-99.000000
25%,0.000000,-99.00000,-99.000000,-99.000000,-99.000000,1.000000,1.000000,34.00000,0.0,0.0,2.000000,0.0,0.000000,-99.000000,1.000000,-99.0,0.000000
50%,0.000000,-99.00000,-99.000000,-99.000000,-99.000000,1.000000,2.000000,40.00000,0.0,0.0,3.000000,0.0,0.000000,-99.000000,1.000000,-99.0,0.000000
75%,0.000000,-99.00000,-99.000000,-99.000000,-99.000000,2.000000,2.000000,46.00000,0.0,0.0,4.000000,0.0,0.000000,-99.000000,2.000000,-99.0,0.000000
max,1.000000,9.00000,1481.000000,7665.000000,7200.000000,7.000000,7.000000,60.00000,0.0,0.0,5.000000,0.0,2.000000,180.000000,2.000000,-99.0,6135.720000


In [12]:
df_train.iloc[:, 3].value_counts()


-99.0      15156
 0.0        1434
 1000.0       40
 500.0        39
 600.0        34
           ...  
 494.0         1
 1329.0        1
 561.0         1
 678.0         1
 1456.0        1
Name: basic_3, Length: 549, dtype: int64

In [13]:
for i in range(2, 17):
    print(df_train.iloc[:, i].value_counts())


-99     15157
 121       30
 124       29
 115       29
 62        27
        ...  
 267        1
 215        1
 238        1
 320        1
 366        1
Name: basic_2, Length: 344, dtype: int64
-99.0      15156
 0.0        1434
 1000.0       40
 500.0        39
 600.0        34
           ...  
 494.0         1
 1329.0        1
 561.0         1
 678.0         1
 1456.0        1
Name: basic_3, Length: 549, dtype: int64
-99.0      15156
 0.0         921
 600.0        50
 500.0        50
 1000.0       47
           ...  
 405.0         1
 1302.0        1
 434.0         1
 786.0         1
 1456.0        1
Name: basic_4, Length: 718, dtype: int64
 1     10562
 2      5352
 3      1260
 0       428
 4       208
-99      160
 5        25
 6         3
 7         1
Name: basic_5, dtype: int64
 1     8738
 2     6380
 3     2157
 4      473
-99     160
 5       77
 6       12
 7        2
Name: basic_6, dtype: int64
38    857
39    808
34    796
37    767
33    765
42    755
41    736
36    722


In [14]:
num_col = ["basic_2",
           "basic_3",
           "basic_4",
           "basic_7",
           "basic_13"]
type_col = ["basic_1",
            "basic_10",
            "basic_12",
            "basic_14"]
drop_col = ["basic_8",
            "basic_9",
            "basic_11",
            "basic_15"]


In [15]:
df_train.drop(columns=drop_col)


,label,basic_1,basic_2,basic_3,basic_4,basic_5,basic_6,basic_7,basic_10,basic_12,basic_13,basic_14,loan1_1
cust_id,,,,,,,,,,,,,
0,0.0,-99,-99,-99.0,-99.0,1,2,37,2,0,-99,1,0.000000
1,0.0,-99,-99,-99.0,-99.0,1,2,49,2,0,-99,1,0.000000
2,0.0,-99,-99,-99.0,-99.0,1,1,52,3,0,-99,2,0.000000
3,0.0,-99,-99,-99.0,-99.0,1,1,54,5,0,-99,1,0.000000
5,0.0,-99,-99,-99.0,-99.0,2,3,33,3,0,-99,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,0.0,1,70,0.0,544.0,2,5,35,1,0,29,1,1.054412
24995,0.0,-99,-99,-99.0,-99.0,2,2,42,3,0,-99,2,0.000000
24997,0.0,-99,-99,-99.0,-99.0,2,2,55,2,0,-99,2,0.000000


In [16]:
from sklearn import model_selection


X = df_train[num_col+type_col]
Y = df_train["label"]
seed = 666
test_size = 0.1
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=test_size, random_state=seed)


In [17]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 6,
    'eta': 0.1,
    'colsample_bytree':0.8,
    'objective': 'binary:logistic',
    #'eval_metric':'logloss'
}

evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)


[0]	eval-logloss:0.61057	train-logloss:0.61057
[1]	eval-logloss:0.54309	train-logloss:0.54302
[2]	eval-logloss:0.48700	train-logloss:0.48691
[3]	eval-logloss:0.43989	train-logloss:0.43974
[4]	eval-logloss:0.39996	train-logloss:0.39970
[5]	eval-logloss:0.36595	train-logloss:0.36553
[6]	eval-logloss:0.33665	train-logloss:0.33619
[7]	eval-logloss:0.31146	train-logloss:0.31086
[8]	eval-logloss:0.28966	train-logloss:0.28895
[9]	eval-logloss:0.27078	train-logloss:0.26994
[10]	eval-logloss:0.25443	train-logloss:0.25343
[11]	eval-logloss:0.24014	train-logloss:0.23900
[12]	eval-logloss:0.22777	train-logloss:0.22650
[13]	eval-logloss:0.21698	train-logloss:0.21552
[14]	eval-logloss:0.20759	train-logloss:0.20596
[15]	eval-logloss:0.19940	train-logloss:0.19756
[16]	eval-logloss:0.19239	train-logloss:0.19023
[17]	eval-logloss:0.18620	train-logloss:0.18387
[18]	eval-logloss:0.18089	train-logloss:0.17834
[19]	eval-logloss:0.17625	train-logloss:0.17343
[20]	eval-logloss:0.17228	train-logloss:0.16912
[2

C:\Users\tassadar667\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:568: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[43]	eval-logloss:0.15051	train-logloss:0.14022
[44]	eval-logloss:0.15053	train-logloss:0.13991
[45]	eval-logloss:0.15050	train-logloss:0.13969
[46]	eval-logloss:0.15046	train-logloss:0.13951
[47]	eval-logloss:0.15036	train-logloss:0.13926
[48]	eval-logloss:0.15048	train-logloss:0.13898
[49]	eval-logloss:0.15052	train-logloss:0.13878
[50]	eval-logloss:0.15045	train-logloss:0.13852
[51]	eval-logloss:0.15051	train-logloss:0.13839
[52]	eval-logloss:0.15053	train-logloss:0.13829
[53]	eval-logloss:0.15058	train-logloss:0.13821
[54]	eval-logloss:0.15070	train-logloss:0.13803
[55]	eval-logloss:0.15066	train-logloss:0.13795
[56]	eval-logloss:0.15080	train-logloss:0.13782
[57]	eval-logloss:0.15076	train-logloss:0.13772
[58]	eval-logloss:0.15080	train-logloss:0.13766
[59]	eval-logloss:0.15081	train-logloss:0.13763
[60]	eval-logloss:0.15077	train-logloss:0.13743
[61]	eval-logloss:0.15081	train-logloss:0.13741
[62]	eval-logloss:0.15087	train-logloss:0.13734
[63]	eval-logloss:0.15087	train-logloss:

In [35]:
df_test=pd.read_csv('data\main\df_test_b1.csv',index_col=0)
temp=xgb.DMatrix(df_test[num_col+type_col])
pred_label=list(bst.predict(temp))
pred_label=[1 if i>=0.2 else 0 for i in pred_label]
res=pd.DataFrame({
    'cust_id':list(df_test.index),
    'label':pred_label,
})
res.to_csv("res.csv",encoding="utf-8",index=False)